# **Pandas en Python:**
> **Nota:** Este notebook está hecho en **Polars** para que puedas comparar **sección por sección** con tu notebook original de Pandas.

## Introducción y explicación en profundidad de **Polars** (y comparación con Pandas)

**Polars** es una librería de DataFrames para Python diseñada para ser **muy rápida** y **eficiente en memoria**. Su núcleo está implementado en **Rust** y muchas operaciones usan un modelo **columnar** (alineado con **Apache Arrow**), lo que suele acelerar transformaciones por columnas.

### Por qué Polars suele ser más rápido
- **Expresiones (vectorización):** transformaciones declarativas con `pl.col(...)`, `pl.when(...)`, etc.
- **Multihilo:** muchas operaciones se paralelizan automáticamente.
- **Modo Lazy (planificador + optimizador):** construye un plan y lo optimiza antes de ejecutar:
  - *Predicate pushdown* (empuja filtros hacia la lectura),
  - *Projection pushdown* (lee solo columnas necesarias),
  - reordenación de operaciones para reducir coste.

### Diferencias importantes con Pandas
- **No hay `Index` como concepto central.** En Polars normalmente mantienes `id` como columna.
- **API distinta:** Polars se parece más a SQL (expresiones) que a Pandas (estilo imperativo).
- **UDFs tipo `apply`** existen, pero se recomiendan evitar en datos grandes (penalizan rendimiento/optimizaciones).

### Cuándo usar cada uno
- **Polars:** datos grandes, pipelines largos, rendimiento.
- **Pandas:** ecosistema enorme, compatibilidad, exploración rápida.
- Mezcla típica: transformar con Polars y, si hace falta, `df.to_pandas()`.


### **Importar librería**


In [1]:
# En Colab, instala Polars si hace falta:
!pip -q install polars

import polars as pl
import random


In [2]:
# (Celda vacía en el original)


### **Carga de Datos**


In [3]:
# CARGA DE DATOS
df = pl.read_csv("./data/dataset.csv") # Por defecto detecta las columnas por el encabezado del archivo
# Puedo especificar algunos parametros adicionales como por ejemplo una columna como indice
# pl.read_csv("./dataset.csv", index_col = "id")

df.head()


id,full_text,favorites,retweets,mentions,country,user,followers,followees
i64,str,i64,i64,i64,str,str,i64,i64
183721,"""Flying home to run down from t…",23,null,10,"""ECUADOR""","""leonardokuffo""",389,258
183722,"""Today we commemorate and MNML …",500,21,null,"""BRASIL""","""mateusmartins""",982,1822
183723,"""Today we have reached US$6.55 …",190,123,6,"""MEXICO""","""pedrojuarez""",12,129
183724,"""Faking It by Joel Atwell. Writ…",131,76,3,"""ECUADOR""","""galocastillo""",332,378
183725,"""Welcome back! 🙌""",113,130,9,"""MEXICO""","""pedrojuarez""",12,129


### **Asignar la columna Id como índice**
> **Diferencia:** en Polars no existe `Index` como en Pandas, por eso `index_col="id"` no aplica.  
> Equivalente práctico: mantener `id` como columna y **ordenar** por `id` (o filtrar por `id`).


In [5]:
# (LÍNEA ORIGINAL EN PANDAS PARA COMPARAR)
# # Puedo especificar algunos parametros adicionales como por ejemplo una columna como indice
# df =pd.read_csv(("./data/dataset.csv"), index_col = "id")

# En Polars no existe index_col/Index. Leemos y ordenamos por 'id' (equivalente práctico).
df = pl.read_csv("./data/dataset.csv")
df = df.sort('id')
df.head()


id,full_text,favorites,retweets,mentions,country,user,followers,followees
i64,str,i64,i64,i64,str,str,i64,i64
183721,"""Flying home to run down from t…",23,null,10,"""ECUADOR""","""leonardokuffo""",389,258
183722,"""Today we commemorate and MNML …",500,21,null,"""BRASIL""","""mateusmartins""",982,1822
183723,"""Today we have reached US$6.55 …",190,123,6,"""MEXICO""","""pedrojuarez""",12,129
183724,"""Faking It by Joel Atwell. Writ…",131,76,3,"""ECUADOR""","""galocastillo""",332,378
183725,"""Welcome back! 🙌""",113,130,9,"""MEXICO""","""pedrojuarez""",12,129


### **Mostrar la cabecera y primeras 8 filas**


In [ ]:
df.head(8)


### **Mostrar las ultimas 8 filas**


In [ ]:
df.tail(8)


### **Proporciona Estadísticas Descriptivas**


In [ ]:
df.describe()


### **Proporciona Estadísticas sobre columnas categóricas o de texto**
> En Pandas: `df.describe(include='all')`.  
> En Polars construimos un resumen útil: `dtype`, `nulls`, `n_unique`, y un top frecuente por columna categórica.


In [ ]:
# Resumen general por columna
resumen = []
for c in df.columns:
    s = df.get_column(c)
    resumen.append({
        "columna": c,
        "dtype": str(s.dtype),
        "nulls": int(s.null_count()),
        "n_unique": int(s.n_unique()),
    })
pl.DataFrame(resumen)


In [ ]:
# Top 5 valores más frecuentes de una columna categórica (ej: country)
df.group_by("country").len().sort("len", descending=True).head(5)


### **Proporciona los nombres de todas las columnas en el DataFrame como un objeto Index de pandas.**


In [ ]:
df.columns


#### **Quitar filas con elemenos NaN.**


In [ ]:
# En Polars: valores ausentes suelen ser null
df_filtrado = df.drop_nulls()
df_filtrado.head()


#### **Llenar los valores NaN con un valor por defecto.**


In [ ]:
df_filtrado_con_valores_por_defecto = df.fill_null(0)
df_filtrado_con_valores_por_defecto.head()


#### **Llenar los valores de una columna que contenga NaN con un valor por defecto.**


In [ ]:
df_filtrado_con_valores_por_defecto_en_columna = df.with_columns([
    pl.col("retweets").fill_null(0),
    pl.col("mentions").fill_null(-1),
    pl.col("favorites").fill_null(0),
])
df_filtrado_con_valores_por_defecto_en_columna.head()


### **Muestra el tipo de dato de cada columna en el DataFrame**


In [ ]:
df_filtrado_con_valores_por_defecto_en_columna.schema


### **Mostrar una columna o mas del dataframe**


In [ ]:
df["favorites"].head(5)
df.select(["favorites","country"]).head(5)


# **Filtrado**


### **Dar las ultimas 5**


In [ ]:
df.tail(5)


### **Dar las primeras 5**


In [ ]:
df.head(5)


### **Dar la primera fila**


In [ ]:
df.head(1)


#### **Filtrar por los Identificadores**


In [ ]:
df.filter(pl.col("id").is_in([183743, 183744])).select(["favorites","country","id"])


## **Filtrado con Condiciones**


#### Obtener los registros donde los favoritos son mayor que 400


In [ ]:
df.filter(pl.col("favorites").fill_null(0) > 400)


#### Obtener los registros donde los favoritos son mayor que 400 y metions sea mayor que 20


In [ ]:
df.filter(
    (pl.col("favorites").fill_null(0) > 400) &
    (pl.col("mentions").fill_null(-1) > 20)
)


#### Obtener los registros que tenga en full_text "Programming"


In [ ]:
df.filter(pl.col("full_text").str.contains("Programming"))


# **Transformación de Datos**
Apartir de los datos se puede hacer limpieza, transformar o calcular nuevas columnas.
> Diferencia: Polars recomienda expresiones vectorizadas. UDFs tipo `apply` (aquí `map_elements`) pueden ser más lentas.


#### Transformacion de datos de una columna
Rellenar `retweets` con 0 y `mentions` con -1, y crear `ganancias`.


In [ ]:
df = df.with_columns([
    pl.col("retweets").fill_null(0),
    pl.col("mentions").fill_null(-1),
    pl.col("favorites").fill_null(0),
])

df = df.with_columns(
    pl.col("retweets")
      .map_elements(lambda x: int(x) * random.randint(3,5), return_dtype=pl.Int64)
      .alias("ganancias")
)

df.select(["id","retweets","ganancias","mentions"]).head(10)


#### Transformación de datos de dos o mas columnas
`popularidad = followees / followers` (con protección si followers==0).


In [ ]:
df = df.with_columns(
    pl.when(pl.col("followers") == 0)
      .then(None)
      .otherwise(pl.col("followees") / pl.col("followers"))
      .alias("popularidad")
)

df.select(["id","followees","followers","popularidad"]).head(10)


### **Transformación agrupación o agregación**


#### Sumar la información de otras columnas mediante la agre...to calcular el promedio de "Me gustas" en los tweet por páis.


In [ ]:
df_mean = (
    df.select(["country","favorites","retweets","mentions","followers","followees","ganancias","popularidad"])
      .group_by("country")
      .mean()
)

df_mean


#### Agrupando por pais y aplicando diferentes funciones de ...lumna (followers: suma , mentions: media y retweets: maximo )


In [ ]:
grouped = (
    df.group_by("country")
      .agg([
          pl.col("followers").sum().alias("followers"),
          pl.col("mentions").mean().alias("mentions"),
          pl.col("retweets").max().alias("retweets"),
      ])
)

grouped


#### Crear un DataFrame llamado "grouped" que contenga la agrupación anterior


In [ ]:
grouped.head()


#### Filtrar los follower de DataFrame anterior donde sean mayores de 5000


In [ ]:
grouped.filter(pl.col("followers") > 5000)


In [ ]:
# (Celda vacía en el original)


#### Guardar nuestros datos transformados a CSV, JSON, SQL, ...


In [ ]:
grouped.write_csv("./data/grouped.csv")
grouped.write_ndjson("./data/grouped.ndjson")


---
## Extra (opcional): versión Lazy equivalente (esto es “extra” respecto a Pandas)


In [8]:
lf = pl.scan_csv("./data/dataset.csv")

resultado_lazy = (
    lf.with_columns([
        pl.col("favorites").fill_null(0),
        pl.col("retweets").fill_null(0),
        pl.col("mentions").fill_null(-1),
    ])
    .filter(pl.col("favorites") > 400)
    .group_by("country")
    .agg(pl.col("followers").sum().alias("followers"))
    .sort("followers", descending=True)
    .collect()
)

resultado_lazy


country,followers
str,i64
"""BRASIL""",2128
"""ECUADOR""",389
"""MEXICO""",42
